#### This Version Creates the Graph in Neo4J then runs the Queries.

In [54]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [55]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_1_8')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-b9410d32-04b1-467e-8f36-644035e8f0e0/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [56]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-b9410d32-04b1-467e-8f36-644035e8f0e0/bin'

# Construct the command
#command = [
#    './bin/neo4j-admin', 'database', 'import', 'full',
#    '--nodes=import/nodes.csv',
#    '--relationships=import/relationships.csv', 'neo4j'
#]

command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
Neo4j version: 5.12.0
Importing the contents of these files into C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-b9410d32-04b1-467e-8f36-644035e8f0e0\data\databases\neo4j:
Nodes:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-b9410d32-04b1-467e-8f36-644035e8f0e0\import\nodes.csv

Relationships:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-b9410d32-04b1-467e-8f36-644035e8f0e0\import\relationships.csv


Available resources:
  Total machine memory: 31.86GiB
  Free machine memory: 8.041GiB
  Max heap memory : 910.5MiB
  Max worker threads: 8
  Configured max memory: 6.535GiB
  High parallel IO: true

Cypher type normalization is enabled (disable with --normalize-types=false):
  Property type of 'WaveNumber' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-b9410d32-04b1-467e-8f36-644035e8f0e0\import\nodes.csv
  Property type of 'Absorbance' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\re

Run and connect to the Neo4j Database

In [58]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [59]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        SAMPLE: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        },
        GRID: {
          orientation: 'UNDIRECTED',
          properties: 'DIST'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [60]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [61]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [62]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [63]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'DIST'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["score"]) for record in results]

In [64]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [65]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [66]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'DIST' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["Community"]) for record in results]

In [67]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [68]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'DIST',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, gds.util.asNode(nodeId).Node_ID AS id, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["id"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [69]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [70]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'id', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'id', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'id', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'id', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'id', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'id', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'id', 'Louvain'])

In [ ]:
pagerank_df = pagerank_df.groupby("name").mean().reset_index()

In [65]:
degree_df = degree_df.groupby("name").mean().reset_index()
eigenvector_df = eigenvector_df.groupby("name").mean().reset_index()
articlerank_df = articlerank_df.groupby("name").mean().reset_index()
eigenvector_df = eigenvector_df.groupby("name").mean().reset_index()
leiden_df = leiden_df.groupby("name").first().reset_index()
louvain_df = louvain_df.groupby("name").first().reset_index()

In [71]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['id', 'name'], how='left')

In [72]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,id,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210211-2-04,1194951,1.828433,1056.449970,0.010088,0.309450,258,9121
1,210304-2-31,2583574,1.759516,815.321280,0.010280,0.278378,246,22031
2,210304-2-31,2583529,1.749199,811.771423,0.010226,0.277744,246,22031
3,210211-1-39,1155420,1.726280,810.093873,0.010269,0.277686,246,22031
4,210211-1-47,1176512,1.724913,798.895393,0.010081,0.275876,246,22031
...,...,...,...,...,...,...,...,...
39803,210504-2-02,6368914,0.231335,25.219108,0.000304,0.154139,78,6468
39804,210114-2-35,486889,0.229591,34.826437,0.000425,0.155596,25,26877
39805,210511-1-00,7022378,0.229094,25.012391,0.000415,0.154187,249,11178
39806,210211-2-31,1264905,0.221288,24.309451,0.000362,0.154034,148,13356


In [73]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'id', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,610,0.046864,-0.019190,-0.212645,0.023161,-0.285587,-0.138393,-0.067084,-0.106714,...,-0.584568,-0.092499,0.037837,0.232219,-0.252768,0.354207,-0.156446,-0.025295,-0.126429,-0.126047
1,201210-1-00,726,0.014531,-0.019809,-0.234477,0.012155,-0.297501,-0.120586,-0.066471,-0.099693,...,-0.558913,-0.108562,0.022787,0.231725,-0.250553,0.335475,-0.156408,-0.023427,-0.125938,-0.119282
2,201210-1-00,1020,-0.506066,0.075603,-0.114797,0.076330,0.343902,-0.125036,0.149199,0.103230,...,-0.289320,0.248113,0.042406,-0.120563,0.140712,-0.077005,-0.025665,-0.089866,0.067022,-0.366269
3,201210-1-00,1177,0.053234,0.217961,0.059547,0.150447,-0.053101,0.274631,-0.012392,-0.284800,...,-0.238510,-0.134622,-0.074236,-0.333366,-0.006053,-0.142567,-0.087524,-0.050380,-0.185141,-0.227133
4,201210-1-00,1817,0.032186,-0.088392,-0.023469,0.017336,-0.087878,-0.013857,-0.102035,0.300177,...,0.120552,-0.049015,0.028629,-0.375053,0.019280,-0.090477,-0.206084,-0.205212,-0.069389,0.067620


Delete the projection

In [87]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [88]:
close_driver()

In [58]:
status_df

,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot,Node_ID:ID
0,201210-1-00,618.90662,1842.5710,Normal,57.732793,181.9741,"(12, 1)",613
1,201210-1-00,696.81091,1851.9185,Normal,33.525178,193.5749,"(13, 1)",727
2,201210-1-00,901.13898,1746.4041,Normal,107.305604,157.9830,"(18, 1)",1026
3,201210-1-00,996.12756,1702.7238,Normal,26.861586,115.9938,"(19, 1)",1165
4,201210-1-00,1542.14120,1475.0056,Normal,15.749470,87.6968,"(30, 1)",1964
...,...,...,...,...,...,...,...,...
51922,210526-3-49,450.79727,1482.5781,Hyperglycemia,11.954149,98.2019,"(9, 1)",8008132
51923,210526-3-49,520.50116,1482.3167,Hyperglycemia,13.875621,86.4581,"(10, 1)",8008234
51924,210526-3-49,705.01141,1507.8912,Hyperglycemia,54.718502,125.7380,"(14, 1)",8008504
51925,210526-3-49,1000.22780,1505.2144,Hyperglycemia,18.241825,110.7786,"(20, 1)",8008936


In [74]:
status_df = pd.read_csv('../../data/graph_1_8/nodes.csv')

In [75]:
fastRP_df['id'] = fastRP_df['id'].astype(int)

In [76]:
df['id'] = df['id'].astype(int)

In [77]:
graph_df = pd.merge(df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
graph_df.drop(columns=["Node_ID:ID"], inplace=True)

In [78]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="id", right_on="Node_ID:ID", how="inner")
fast_graph_df.drop(columns=["Node_ID:ID"], inplace=True)

In [79]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)
graph_df.drop(columns=['SpecID_y','Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)

In [80]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [81]:
fast_graph_df.drop(columns=['SpecID_y', 'WaveNumber:float', 'Absorbance:float', 'Widths:float', 'Prominences:float', 'GridSlot'], inplace=True)
fast_graph_df.rename(columns={'SpecID_x':'SpecID'}, inplace=True)

In [82]:
fast_graph_df

,SpecID,id,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status
0,201210-1-00,610,0.046864,-0.019190,-0.212645,0.023161,-0.285587,-0.138393,-0.067084,-0.106714,...,-0.092499,0.037837,0.232219,-0.252768,0.354207,-0.156446,-0.025295,-0.126429,-0.126047,Normal
1,201210-1-00,726,0.014531,-0.019809,-0.234477,0.012155,-0.297501,-0.120586,-0.066471,-0.099693,...,-0.108562,0.022787,0.231725,-0.250553,0.335475,-0.156408,-0.023427,-0.125938,-0.119282,Normal
2,201210-1-00,1020,-0.506066,0.075603,-0.114797,0.076330,0.343902,-0.125036,0.149199,0.103230,...,0.248113,0.042406,-0.120563,0.140712,-0.077005,-0.025665,-0.089866,0.067022,-0.366269,Normal
3,201210-1-00,1177,0.053234,0.217961,0.059547,0.150447,-0.053101,0.274631,-0.012392,-0.284800,...,-0.134622,-0.074236,-0.333366,-0.006053,-0.142567,-0.087524,-0.050380,-0.185141,-0.227133,Normal
4,201210-1-00,1817,0.032186,-0.088392,-0.023469,0.017336,-0.087878,-0.013857,-0.102035,0.300177,...,-0.049015,0.028629,-0.375053,0.019280,-0.090477,-0.206084,-0.205212,-0.069389,0.067620,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39803,210526-1-47,7832749,0.126270,0.104700,-0.084292,0.059099,0.005476,-0.327246,-0.278872,0.049709,...,0.148207,-0.189241,0.054733,0.324306,0.380865,-0.104974,-0.250026,0.064236,-0.151102,Hyperglycemia
39804,210526-1-47,7832977,0.240517,-0.003918,0.016701,0.023759,0.162307,-0.379389,-0.129736,-0.259684,...,0.357598,0.058129,0.081749,-0.042940,0.246318,-0.413193,-0.288141,0.208139,-0.122411,Hyperglycemia
39805,210526-1-47,7833190,0.076939,-0.032755,-0.175425,-0.007901,-0.019389,-0.089740,-0.024492,-0.188793,...,0.272912,0.039550,0.070570,0.076701,0.026609,0.046324,0.092584,-0.092233,-0.463957,Hyperglycemia
39806,210526-1-47,7833365,-0.003860,-0.136335,-0.208623,-0.091058,0.038933,-0.531144,-0.189479,0.143440,...,0.196622,0.116839,-0.004043,-0.294973,-0.203124,-0.102488,0.060309,-0.068278,-0.138447,Hyperglycemia


In [83]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"

# Set the bin size
bin_size = 25

# Add a "Bin" column to the DataFrame
graph_df['Bin'] = graph_df['WaveNumber:float'].apply(lambda x: calculate_bin_interval(x, bin_size))

In [84]:
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = graph_df.pivot_table(index='SpecID', columns='Bin', values=['PageRank', 'DegreeCentrality', 'EigenvectorCentrality', 'ArticleRank', 'Leiden', 'Louvain'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = graph_df[['SpecID', 'Status']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)

C:\Users\stang\AppData\Local\Temp\ipykernel_9380\286844324.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  peak_bins.fillna(False, inplace=True)


In [85]:
graph_bins = peak_bins.reset_index()

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = peak_bins.drop(['Status'], axis=1)
y = peak_bins['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.7781 +/- 0.0268
ExtraTreesClassifier Cross-Validation Accuracy: 0.7668 +/- 0.0227


In [87]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [88]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [89]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [90]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.7703826955074875


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.79      0.71      0.75       202
 Hypoglycemia       0.76      0.82      0.79       197
       Normal       0.76      0.78      0.77       202

     accuracy                           0.77       601
    macro avg       0.77      0.77      0.77       601
 weighted avg       0.77      0.77      0.77       601


Confusion Matrix:
[[144  27  31]
 [ 18 161  18]
 [ 21  23 158]]


In [91]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.7637271214642263


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.79      0.75      0.77       202
 Hypoglycemia       0.73      0.79      0.76       197
       Normal       0.78      0.75      0.77       202

     accuracy                           0.76       601
    macro avg       0.77      0.76      0.76       601
 weighted avg       0.77      0.76      0.76       601


Confusion Matrix:
[[151  28  23]
 [ 21 156  20]
 [ 19  31 152]]


In [92]:
get_feature_importances(rf_model, X)

,Feature,Importance
115,DegreeCentrality_275-300,0.013582
204,EigenvectorCentrality_700-725,0.013009
331,Louvain_275-300,0.012238
43,ArticleRank_275-300,0.012192
403,PageRank_275-300,0.011798
187,EigenvectorCentrality_275-300,0.011047
420,PageRank_700-725,0.010786
259,Leiden_275-300,0.009940
132,DegreeCentrality_700-725,0.009775
60,ArticleRank_700-725,0.009722


In [93]:
get_feature_importances(et_model, X)

,Feature,Importance
43,ArticleRank_275-300,0.012731
403,PageRank_275-300,0.009999
204,EigenvectorCentrality_700-725,0.009658
60,ArticleRank_700-725,0.008993
115,DegreeCentrality_275-300,0.008539
187,EigenvectorCentrality_275-300,0.007920
331,Louvain_275-300,0.007781
420,PageRank_700-725,0.007746
132,DegreeCentrality_700-725,0.007212
259,Leiden_275-300,0.007101


## FastRp

In [94]:
fast_graph_df.drop(columns=['id'], inplace=True)
fast_graph_df = fast_graph_df.groupby(['SpecID', 'Status']).max().reset_index()

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['SpecID', 'Status'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.6056572379367721
Extra Trees Accuracy: 0.6339434276206323


Fast rp mixed with traditional graph metrics

In [96]:
joined_graph = pd.merge(graph_bins, fast_graph_df, on=['SpecID', 'Status'], how='inner')

In [97]:
joined_graph

,SpecID,ArticleRank_1000-1025,ArticleRank_1025-1050,ArticleRank_1050-1075,ArticleRank_1075-1100,ArticleRank_1100-1125,ArticleRank_1125-1150,ArticleRank_1150-1175,ArticleRank_1175-1200,ArticleRank_1200-1225,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,0.337834,False,False,False,False,False,False,False,False,...,0.120552,0.248113,0.042406,0.232219,0.140712,0.354207,0.168510,0.269725,0.067022,0.067620
1,201210-1-01,0.353979,False,False,False,False,False,False,False,False,...,0.198482,0.112457,0.229592,0.274950,0.341363,0.356429,0.265111,0.222204,0.213168,0.099613
2,201210-1-02,False,0.305033,False,False,False,0.371674,False,False,False,...,0.201333,0.329068,0.384762,0.277555,0.340641,0.382444,0.232253,0.220375,0.156532,-0.010695
3,201210-1-03,False,False,False,False,False,False,0.178372,False,False,...,0.373603,0.341958,0.127300,0.203304,0.379320,0.157114,0.209466,0.215301,0.248947,0.058754
4,201210-1-04,0.353831,False,False,False,False,False,False,0.312912,False,...,0.199683,0.212319,0.228004,0.416055,0.339357,0.339386,0.214903,0.221253,0.211021,0.312163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,210526-3-45,0.3604,False,False,False,False,False,False,False,False,...,0.199389,0.143163,0.283524,0.414284,0.329817,0.389216,0.204660,0.265106,0.201229,0.312829
2998,210526-3-46,0.360554,False,False,False,False,False,False,False,False,...,0.199863,0.001205,0.283982,0.163361,0.075377,0.302652,0.204374,0.271440,0.199742,0.055874
2999,210526-3-47,0.359552,False,False,False,False,False,False,False,False,...,0.199839,0.143179,0.283935,0.414573,0.330184,0.389560,0.205154,0.157843,0.200735,0.314063
3000,210526-3-48,False,False,False,False,False,False,False,False,False,...,0.181800,0.142975,0.283615,0.414111,0.330738,0.389792,0.206290,0.156621,0.202089,0.312392


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.7470881863560732
Extra Trees Accuracy: 0.7587354409317804


In [99]:
# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status'])

# Target variable
y = joined_graph['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.7412 +/- 0.0214
ExtraTreesClassifier Cross-Validation Accuracy: 0.7695 +/- 0.0198
